In [112]:
import os
import math
import numpy as np
import pandas as pd
import datetime
import time
import random
from datetime import date
import pandas_ta as ta
from ta.volatility import KeltnerChannel
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from yahoo_fin import stock_info as si
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [113]:
def load_historic_data(symbol):
    today = datetime.date.today()
    today_str = today.strftime("%Y-%m-%d")
    #  Get last year's data
    start_date = today - (251 * US_BUSINESS_DAY)
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
    # Download data from Yahoo Finance
    try:
        df = si.get_data(symbol, start_date=start_date_str, end_date=today_str, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbol)
        return None

In [114]:
def calculate_keltner_channels(df):
    # Initialize Keltner Channel Indictor
    indicator_keltner = KeltnerChannel(high=df['high'], low=df['low'], close=df["adjclose"], window=20)

    # Add Keltner Channel features
    df['keltner_mband'] = indicator_keltner.keltner_channel_mband()
    df['keltner_hband'] = indicator_keltner.keltner_channel_hband()
    df['keltner_lband'] = indicator_keltner.keltner_channel_lband()
    return df

In [115]:
def apply_strategy_rules(df):
    #  Entry Rule 1: Price breaks through the high channel
    df['keltner_entry_signal'] = np.where((df["adjclose"] >= df["keltner_hband"]) & (df["adjclose"].shift() < df["keltner_hband"]), 1, 0)
    
    #  Exit rule: Price drops below the mid channel
    df['keltner_exit_signal'] = np.where((df["adjclose"] <= df["keltner_mband"]) & (df["adjclose"].shift() > df["keltner_mband"]), 1, 0)

    return df

In [116]:
def execute_strategy(df):
    close_prices = df['adjclose']
    keltner_entry_signals = df['keltner_entry_signal']
    keltner_exit_signals = df['keltner_exit_signal']
    entry_prices = []
    exit_prices = []
    entry_signal = 0
    exit_signal = 0
    hold = 0
    
    for i in range(len(close_prices)):
        #  Check entry and exit signals
        if keltner_entry_signals[i] == 1:
            entry_signal = 1
        else:
            entry_signal = 0
        if keltner_exit_signals[i] == 1:
            exit_signal = 1
        else:
            exit_signal = 0
            
        #  Add entry prices
        if hold == 0 and entry_signal == 1:
            buy_price = close_prices[i]
            entry_prices.append(close_prices[i])
            exit_prices.append(np.nan)  
            entry_signal = 0
            hold = 1
        #  Evaluate exit strategy
        elif hold == 1 and exit_signal == 1:
            entry_prices.append(np.nan)
            exit_prices.append(close_prices[i]) 
            exit_signal = 0
            hold = 0
        else:
            #  Neither entry nor exit
            entry_prices.append(np.nan) 
            exit_prices.append(np.nan) 
            
    return entry_prices, exit_prices

In [117]:
def plot_graph(df, entry_prices, exit_prices):
    keltner_high = df['keltner_hband']
    keltner_mid = df['keltner_mband']
    keltner_low = df['keltner_lband']
    fig = make_subplots(rows=1, cols=1)

    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['adjclose'], line=dict(color="blue", width=1), name="Close"), row = 1, col = 1)
    
    #  Plot Keltner Channels
    fig.add_trace(go.Line(x = df.index, y = keltner_high, line=dict(color="#ffdf80", width=1), name="KC High"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = keltner_mid, line=dict(color="#ffd866", width=1), name="KC Mid"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = keltner_low, line=dict(color="#ffd24d", width=1), name="KC Low"), row = 1, col = 1)
    
    #  Add buy and sell indicators
    fig.add_trace(go.Scatter(x=df.index, y=entry_prices, marker_symbol="arrow-up", marker=dict(
        color='green',
    ),mode='markers',name='Buy'))
    fig.add_trace(go.Scatter(x=df.index, y=exit_prices, marker_symbol="arrow-down", marker=dict(
        color='red'
    ),mode='markers',name='Sell'))
    
    fig.update_layout(
        title={'text':'Keltner Channel', 'x':0.5},
        autosize=False,
        width=900,height=600)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()
    

In [118]:
def calculate_buy_hold_profit(investment, df):
    close_prices = df['close']
    buy_quantity = investment / close_prices[0]
    sell_amount = buy_quantity * close_prices[len(close_prices)-1]
    profit = sell_amount - investment
    return profit

In [119]:
def calculate_strategy_profit(investment, entry_prices, exit_prices):
    entry_price = 0
    hold = 0
    total_profit = 0
    quantity = 0
    available_funds = investment
    purchase_amount = 0
    
    for i in range(len(entry_prices)):
        current_entry_price = entry_prices[i]
        current_exit_price = exit_prices[i]
        
        if not math.isnan(current_entry_price) and hold == 0:
            entry_price = current_entry_price
            quantity = available_funds / entry_price
            purchase_amount = quantity * entry_price
            hold = 1
        elif hold == 1 and not math.isnan(current_exit_price):
            hold = 0
            proceeds = quantity * current_exit_price
            profit_or_loss = proceeds - purchase_amount
            available_funds = available_funds + profit_or_loss
            total_profit += profit_or_loss
        
    return total_profit

In [120]:
#  Perform analysis
investment = 1000
df = load_historic_data('PTON')
df.reset_index(inplace=True)
df = calculate_keltner_channels(df)
df = apply_strategy_rules(df)
entry_prices, exit_prices = execute_strategy(df)
profit_or_loss = calculate_strategy_profit(investment, entry_prices, exit_prices)
buy_hold_profit = calculate_buy_hold_profit(investment, df)
plot_graph(df, entry_prices, exit_prices)

C:\Users\dutch\anaconda3\envs\tradesystem1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [121]:
def perform_analysis(symbol, df, investment):
    df = df.reset_index()
    df = calculate_keltner_channels(df)
    df = apply_strategy_rules(df)
    
    entry_prices, exit_prices = execute_strategy(df)
    profit_or_loss = calculate_strategy_profit(investment, entry_prices, exit_prices)
    buy_hold_profit = calculate_buy_hold_profit(investment, df)
    return profit_or_loss, buy_hold_profit

In [122]:
# Backtesting using NASDAQ 100
nasdaq_100_df = pd.read_csv('https://raw.githubusercontent.com/justmobiledev/python-algorithmic-trading/main/data/nasdaq_100.csv')
nasdaq_100 = nasdaq_100_df['Symbol'].to_numpy()

In [123]:
#  Backtesting
total_strategy_profit = 0
total_buy_hold_profit = 0
for symbol in nasdaq_100:
    df = load_historic_data(symbol)
    if df is None or df.empty:
        continue
    df.reset_index(inplace=True)
    
    #  Random interval between remote fetch to avoid spam issues
    random_secs = random.uniform(0, 1)
    time.sleep(random_secs)
    
    #  Run backtest
    profit, buy_hold_profit = perform_analysis(symbol, df, investment=investment) 
    print(f"Backtest profit for symbol {symbol}: ${math.trunc(profit)}, buy & hold: ${math.trunc(buy_hold_profit)}")
    total_strategy_profit += profit
    total_buy_hold_profit += buy_hold_profit
  
print(f"\nAvg strategy profit per stock: ${math.trunc(total_strategy_profit / len(nasdaq_100))}")
print(f"\nAvg buy & hold profit per stock: ${math.trunc(total_buy_hold_profit / len(nasdaq_100))}")

Backtest profit for symbol AAPL: $168, buy & hold: $226
Backtest profit for symbol ABNB: $-115, buy & hold: $-85
Backtest profit for symbol ADBE: $142, buy & hold: $-194
Backtest profit for symbol ADI: $-124, buy & hold: $0
Backtest profit for symbol ADP: $204, buy & hold: $147
Backtest profit for symbol ADSK: $-59, buy & hold: $-349
Backtest profit for symbol AEP: $138, buy & hold: $139
Backtest profit for symbol ALGN: $-244, buy & hold: $-414
Backtest profit for symbol AMAT: $-305, buy & hold: $-143
Backtest profit for symbol AMD: $31, buy & hold: $114
Backtest profit for symbol AMGN: $15, buy & hold: $-18
Backtest profit for symbol AMZN: $5, buy & hold: $-127
Backtest profit for symbol ANSS: $9, buy & hold: $-242
Backtest profit for symbol ASML: $-41, buy & hold: $-61
Backtest profit for symbol ATVI: $-16, buy & hold: $-161
Backtest profit for symbol AVGO: $32, buy & hold: $289
Backtest profit for symbol BIDU: $-385, buy & hold: $-466
Backtest profit for symbol BIIB: $18, buy & hold